In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpu-cluster"
vm_size = 'STANDARD_NC6'
max_nodes = 3

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    # create the cluster and wait for completion
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

In [ ]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data/mnist', target_path='mnist', show_progress=True)

In [ ]:
ds_data = ds.as_mount()

In [ ]:
from azureml.train.dnn import TensorFlow

script_params={
'--data-dir': ds_data
}

estimator= TensorFlow(
  source_directory='./scripts',
  compute_target=compute_target,
  script_params=script_params,
  framework_version='1.13',
  entry_script='train.py')